Classifying the 102 Category Flower Dataset with Caffe
===========================================


Before we begin, you should follow the instructions in the README. This will show you how to train a caffe neural net on the Oxford 102 Category Flowers data set, found [here](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/).

next, set `CAFFE_ROOT` below to the root of your caffe installation.

In [26]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Make sure that caffe is on the python path:
CAFFE_ROOT = '/home/waylonflinn/Development/caffe/'
import sys
sys.path.insert(0, CAFFE_ROOT + 'python')

import caffe

from caffe.io import load_image
from scipy.io import loadmat

# Set the right path to your model definition file, pretrained model weights,
# and the image you would like to classify.
REF_MODEL_FILE = CAFFE_ROOT + 'models/bvlc_reference_caffenet/deploy.prototxt'
REF_PRETRAINED = CAFFE_ROOT + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

MODEL_FILE = './deploy.prototxt'
PRETRAINED = './oxford102_iter_50000.caffemodel'

RAW_DATA_DIR = './data/'
IMAGE_DATA_DIR = RAW_DATA_DIR + 'oxford102/jpg/'
SEGMENT_DATA_DIR = RAW_DATA_DIR + 'oxford102/segmim/'
LABEL_FILE = RAW_DATA_DIR + 'imagelabels.mat'
# training, test and validation set indeces
SET_FILE = RAW_DATA_DIR + 'setid.mat'

IMAGE_FORMAT = 'image_{0:05d}.jpg'
SEGMENT_FORMAT = 'segmim_{0:05d}.jpg'

Run this to get bigger plots

In [39]:
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
matplotlib.rcParams['savefig.dpi']

### Import Labels and Set Definitions

In [27]:
flower_mat = loadmat(LABEL_FILE)
labels = flower_mat['labels'][0]

set_mat = loadmat(SET_FILE)
# we used test data for training, so swap that here
test_ids = set_mat['trnid'][0]
train_ids = set_mat['tstid'][0]
valid_ids = set_mat['valid'][0]

In [4]:
labels

In [28]:
test_ids

Map the label to the name. This is only a partial list, which I've constructed by hand. I haven't, yet, found a complete mapping of labels to names.

This page is good for adding new names: [Categories](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/categories.html)

In [6]:
name_map = { 
            1 : 'Pink Primrose',
            2 : 'Hard-Leaved Pocket Orchid',
            3 : 'Canterbury Bells',
            4 : 'Sweet Pea',
            5 : 'English Marigold',
            6 : 'Tiger Lily',
            7 : 'Moon Orchid',
            8 : 'Bird of Paradise',
            9 : 'Monkshood',
            10: 'Globe Thistle',
            11: 'Snapdragon',
            12: 'Colts Foot',
            13: 'King Protea'}

We'll use the same means as in the caffe classification example (on which this is based).

We'll run on the CPU for the computation.

In [7]:
imagenet_mean = np.load(CAFFE_ROOT + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)

In [8]:
caffe.set_mode_cpu()

In [9]:
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       mean=imagenet_mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

### Search for Images by Label

Let's find all the images with the label `7`. This should be the Moon Orchid.

In [34]:
search_label = 7
search_label_indeces = np.nonzero(labels == search_label)[0]

Should find `40` images

In [35]:
len(search_label_indeces)

## Predictions on the Training Set

### Construct Filename and Load Image

Now we'll load the first image that matches our choosen label.

In [83]:
search_file_index = search_label_indeces[0]

In [85]:
# add one to index because image file numbers start at 1
file_name = IMAGE_FORMAT.format(search_file_index + 1)
test_image = IMAGE_DATA_DIR + file_name
test_image

In [86]:
input_image = load_image(test_image)
plt.imshow(input_image)

Time to classify. The default is to actually do 10 predictions, cropping the center and corners of the image as well as their mirrored versions, and average over the predictions:

In [46]:
output = net.predict([input_image])  # predict takes any number of images, and formats them for the Caffe net automatically
predictions = output[0]
print('prediction shape: {0}'.format(predictions.shape))

Since we're using Softmax, the output (stored in `predictions`) should be a probability distribution over the class labels.

In [47]:
classes = np.arange(0, len(predictions))
plt.bar(classes, predictions, 1.0)
# class numbers start at 1
predicted_class_index = predictions.argmax()
predicted_class = predicted_class_index + 1
predicted_name = name_map[predicted_class] if predicted_class in name_map else '<class only> ->'
print('predicted class: {0} ({1})'.format(predicted_name, predicted_class))
print('probability: {0:.1f}%'.format(predictions[predicted_class_index] * 100))

The predicted class is 7 ('Moon Orchid'), as expected and should be pretty close to a 100% match (since it's in the training set).

## Predictions on Test Set

Now let's try something a little harder. Predicting something from the test set. 

We'll pick the first index from the test set (you should come back and try more stuff later).

In [79]:
test_file_index = test_ids[0]
labels[test_file_index]

First index has a label of `1` ('Pink Primrose'). So that's what we should see in the prediction.
Let's take a look at the image.

In [80]:
# add one to index because image file numbers start at 1
file_name = IMAGE_FORMAT.format(test_file_index + 1)
test_image = IMAGE_DATA_DIR + file_name
input_image = load_image(test_image)
plt.imshow(input_image)

Now we'll make the new prediction for this image. Turning off the oversample feature seems to give better results. You should experiment and see what happens.

In [81]:
output = net.predict([input_image], oversample=False) 
predictions = output[0]
print('prediction shape: {0}'.format(predictions.shape))

In [82]:
classes = np.arange(0, len(predictions))
plt.bar(classes, predictions, 1.0)
# class numbers start at 1
predicted_class_index = predictions.argmax()
predicted_class = predicted_class_index + 1
predicted_name = name_map[predicted_class] if predicted_class in name_map else '<class only> ->'
print('predicted class: {0} ({1})'.format(predicted_name, predicted_class))
print('probability: {0:.1f}%'.format(predictions[predicted_class_index] * 100))

Class label `1` ('Pink Primrose') as expected. You can also see some (much smaller) secondary predictions in the bar chart above (they're tiny, look near the axis).

## Conclusion

Stuff to try:

* check out the `predictions` array
* see what happens when you turn oversampling back on for images in the test set
* make predictions for other images in the test set